In [4]:
import pandas as pd
import numpy as np

# 파일명은 실제 작업 환경에 맞게 수정
승하차_파일 = "data/결과/승하차/통합/1호선_승하차인원_통합.csv"
혼잡도_파일 = "data/결과/승하차/통합/1호선_혼잡도_통합.csv"
시간표_파일 = "data/결과/운행시간표/통합/1호선_열차운행시각표.csv"

승하차_df = pd.read_csv(승하차_파일, encoding="euc-kr")
혼잡도_df = pd.read_csv(혼잡도_파일, encoding="euc-kr")
시간표_df = pd.read_csv(시간표_파일, encoding="euc-kr")

FileNotFoundError: [Errno 2] No such file or directory: 'data/결과/승하차/통합/1호선_승하차인원_통합.csv'

In [ ]:
# 시간 칼럼 자동 추출
시간칼럼_승하차 = [col for col in 승하차_df.columns if ":" in col]
시간칼럼_혼잡도 = [col for col in 혼잡도_df.columns if ":" in col]

# 롱포맷 변환
승차_long = 승하차_df[승하차_df["구분"] == "승차"].melt(
    id_vars=["연번", "평일주말", "호선", "역번호", "역명", "구분"],
    value_vars=시간칼럼_승하차,
    var_name="시간",
    value_name="승차인원",
)
하차_long = 승하차_df[승하차_df["구분"] == "하차"].melt(
    id_vars=["연번", "평일주말", "호선", "역번호", "역명", "구분"],
    value_vars=시간칼럼_승하차,
    var_name="시간",
    value_name="하차인원",
)

# 승/하차 데이터 병합
승하차_merge = pd.merge(
    승차_long.drop(columns="구분"),
    하차_long.drop(columns="구분"),
    on=["연번", "평일주말", "호선", "역번호", "역명", "시간"],
    how="outer",
).fillna(0)
승하차_merge["승차인원"] = 승하차_merge["승차인원"].astype(int)
승하차_merge["하차인원"] = 승하차_merge["하차인원"].astype(int)

# 혼잡도 롱포맷
혼잡도_long = 혼잡도_df.melt(
    id_vars=["연번", "평일주말", "호선", "역번호", "역명", "구분"],
    value_vars=시간칼럼_혼잡도,
    var_name="시간",
    value_name="혼잡도",
)

In [ ]:
import datetime

# string → datetime 변환
시간표_df["열차도착시간"] = pd.to_datetime(시간표_df["열차도착시간"], format="%H:%M:%S")
시간표_df = 시간표_df.sort_values(["호선", "열차코드", "열차도착시간"])

In [ ]:
def get_closest_info(df, 시각, 평일주말, 역명, 호선, cols):
    subset = df[
        (df["호선"] == 호선) & (df["역명"] == 역명) & (df["평일주말"] == 평일주말)
    ].copy()
    try:
        subset["시간_dt"] = pd.to_datetime(
            subset["시간"], format="%H:%M", errors="coerce"
        )
    except:
        subset["시간_dt"] = pd.to_datetime(
            "2024-01-01" + " " + subset["시간"],
            format="%Y-%m-%d %H:%M",
            errors="coerce",
        )
    subset = subset[subset["시간_dt"] <= 시각]
    if subset.empty:
        return [0] * len(cols)
    closest = subset.iloc[-1]
    return [closest[c] for c in cols]


예시_호선 = 1
예시_열차코드 = "K101"

열차경로 = []
누적_인원 = 0

df_특정열차 = 시간표_df[
    (시간표_df["호선"] == 예시_호선) & (시간표_df["열차코드"] == 예시_열차코드)
]
for _, row in df_특정열차.iterrows():
    역명, 은_평일주말, 도착시, 호 = (
        row["역명"],
        row["평일주말"],
        row["열차도착시간"],
        row["호선"],
    )
    승차, 하차 = get_closest_info(
        승하차_merge, 도착시, 은_평일주말, 역명, 호, ["승차인원", "하차인원"]
    )
    혼잡도 = get_closest_info(혼잡도_long, 도착시, 은_평일주말, 역명, 호, ["혼잡도"])[0]
    누적_인원 = max(0, 누적_인원 + int(승차) - int(하차))
    열차경로.append(
        {
            "역명": 역명,
            "도착시간": 도착시,
            "평일주말": 은_평일주말,
            "승차인원": 승차,
            "하차인원": 하차,
            "혼잡도": 혼잡도,
            "누적열차내인원": 누적_인원,
        }
    )

열차경로_df = pd.DataFrame(열차경로).dropna(subset=["혼잡도"])

In [ ]:
from sklearn.preprocessing import MinMaxScaler


def create_sequences(df, time_steps=3):
    features = df[["승차인원", "하차인원", "혼잡도"]].values
    target = df["누적열차내인원"].values
    X, y = [], []
    for i in range(len(features) - time_steps):
        X.append(features[i : i + time_steps])
        y.append(target[i + time_steps])
    return np.array(X), np.array(y)


TIME_STEPS = 3
X, y = create_sequences(열차경로_df, time_steps=TIME_STEPS)

scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
X_reshape = X.reshape(-1, X.shape[2])
X_scaled = scaler_X.fit_transform(X_reshape).reshape(X.shape)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

In [ ]:
# 텐서플로 설치 필요 (import tensorflow)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential()
model.add(LSTM(64, activation="relu", input_shape=(TIME_STEPS, X.shape[2])))
model.add(Dense(32, activation="relu"))
model.add(Dense(1))
model.compile(optimizer="adam", loss="mse")
model.fit(X_scaled, y_scaled, epochs=50, batch_size=8)

In [ ]:
import matplotlib.pyplot as plt

y_pred_scaled = model.predict(X_scaled)
y_pred = scaler_y.inverse_transform(y_pred_scaled)

plt.figure(figsize=(10, 4))
plt.plot(y, label="실제 누적 인원수")
plt.plot(y_pred.flatten(), label="예측 누적 인원수")
plt.legend()
plt.title("LSTM 누적 열차 내 인원수 예측")
plt.xlabel("구간(시간 순서)")
plt.ylabel("인원수")
plt.tight_layout()
plt.show()